In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

# Veri setini yükleme
df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')

# Veri setinin ilk birkaç satırını inceleme
df.head()


In [ ]:
import pandas as pd
from textblob import TextBlob
import string

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Noktalama işaretleri sayısını özellik olarak eklemek
df['response_a_punctuation_count'] = df['response_a'].apply(lambda x: sum([1 for char in x if char in string.punctuation]))
df['response_b_punctuation_count'] = df['response_b'].apply(lambda x: sum([1 for char in x if char in string.punctuation]))

# Kelime çeşitliliği özelliği eklemek
df['response_a_unique_words'] = df['response_a'].apply(lambda x: len(set(x.split())))
df['response_b_unique_words'] = df['response_b'].apply(lambda x: len(set(x.split())))

# Stop words oranı özelliği eklemek (örnek bir stop words listesi kullanıyoruz)
stop_words = set(['the', 'and', 'is', 'in', 'at', 'of', 'it', 'to'])
df['response_a_stop_words_ratio'] = df['response_a'].apply(lambda x: len([word for word in x.lower().split() if word in stop_words]) / len(x.split()))
df['response_b_stop_words_ratio'] = df['response_b'].apply(lambda x: len([word for word in x.lower().split() if word in stop_words]) / len(x.split()))

# Cümlenin baş harf büyüklüğü özelliği eklemek
df['response_a_startswith_upper'] = df['response_a'].apply(lambda x: 1 if x[0].isupper() else 0)
df['response_b_startswith_upper'] = df['response_b'].apply(lambda x: 1 if x[0].isupper() else 0)

# Cümlenin uzunluğu ve farkı özelliği eklemek
df['response_a_length'] = df['response_a'].apply(len)
df['response_b_length'] = df['response_b'].apply(len)
df['response_length_difference'] = df['response_a_length'] - df['response_b_length']

# Kazanan model sütunu eklemek
df['winner'] = df.apply(lambda row: 'model_a' if row['winner_model_a'] == 1 else 'model_b' if row['winner_model_b'] == 1 else 'tie', axis=1)

# Model adı sütunu eklemek
df['model_name'] = df.apply(lambda row: row['model_a'] if row['winner'] == 'model_a' else row['model_b'] if row['winner'] == 'model_b' else '', axis=1)

# One-hot encoding işlemi
encoder = OneHotEncoder()
winner_encoded = encoder.fit_transform(df[['winner']])

# Encode edilmiş sütunları dataframe'e ekleme
df_encoded = pd.concat([df, pd.DataFrame(winner_encoded.toarray(), columns=encoder.categories_[0])], axis=1)

# Sonuçları gösterme



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Veri setini hazırlama fonksiyonu
def prepare_data_for_ml(df):
    # Punctuation count fonksiyonu
    def count_punctuation(text):
        return sum(1 for char in str(text) if char in '.,;:!?')

    # Yeni sütunlar ekleme
    df['response_a_punctuation_count'] = df['response_a'].apply(count_punctuation)
    df['response_b_punctuation_count'] = df['response_b'].apply(count_punctuation)
    df['response_b_startswith_upper'] = df['response_b'].apply(lambda x: int(str(x)[0].isupper()))
    df['response_a_length'] = df['response_a'].str.len()
    df['response_b_length'] = df['response_b'].str.len()
    df['response_length_difference'] = abs(df['response_a_length'] - df['response_b_length'])

    # winner_encoded sütunu oluşturma
    le = LabelEncoder()
    df['winner_encoded'] = le.fit_transform(df['winner'])

    # model_name sütunu oluşturma
    df['model_name'] = np.where(df['winner_model_a'] == 1, df['model_a'], df['model_b'])

    # Kategorik değişkenleri encode etme
    categorical_columns = ['model_a', 'model_b', 'model_name']
    for col in categorical_columns:
        le = LabelEncoder()
        df[f'{col}_encoded'] = le.fit_transform(df[col])

    # Boolean değerleri 0 ve 1'e çevirme
    df['model_a_win'] = df['winner'] == 'model_a'
    df['model_b_win'] = df['winner'] == 'model_b'
    df['winner_tie'] = df['winner'] == 'tie'
    df['tie'] = df['winner'] == 'tie'

    boolean_columns = ['response_b_startswith_upper', 'model_a_win', 'model_b_win', 'winner_tie', 'tie']
    for col in boolean_columns:
        df[col] = df[col].astype(int)

    return df

# Veri setini hazırlama
prepared_df = prepare_data_for_ml(df)

# Gerekli sütunları seçme ve eksik sütunları kontrol etme
columns_to_keep = [
    'id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b',
    'winner', 'model_a_win', 'model_b_win', 'winner_tie',
    'response_a_punctuation_count', 'response_b_punctuation_count',
    'response_b_startswith_upper', 'response_a_length', 'response_b_length',
    'response_length_difference', 'winner_encoded',
    'model_name', 'tie', 'model_a_encoded', 'model_b_encoded', 'model_name_encoded'
]

prepared_df = prepared_df[columns_to_keep]

In [ ]:
prepared_df.head()

In [ ]:
# Veriyi hazırlama
X = prepared_df[['response_a_punctuation_count', 'response_b_punctuation_count',
        'response_a_startswith_upper', 'response_b_startswith_upper',
        'response_a_length', 'response_b_length', 'response_length_difference']]
y = prepared_df['winner_encoded']  # LabelEncoded kullanılarak kodlanmış hedef değişken

# Veriyi train ve test setlerine bölelim
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Veriyi standardize edelim
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modeli eğitme
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# Model üzerinde tahmin yapma
y_pred = model.predict(X_test_scaled)

# Sonuçları değerlendirme
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=['model_a', 'model_b', 'tie']))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Veri setini hazırlama fonksiyonu
def prepare_data_for_ml(df):
    # Punctuation count fonksiyonu
    def count_punctuation(text):
        return sum(1 for char in str(text) if char in '.,;:!?')

    # Yeni sütunlar ekleme
    df['response_a_punctuation_count'] = df['response_a'].apply(count_punctuation)
    df['response_b_punctuation_count'] = df['response_b'].apply(count_punctuation)
    df['response_b_startswith_upper'] = df['response_b'].apply(lambda x: int(str(x)[0].isupper()))
    df['response_a_startswith_upper'] = df['response_a'].apply(lambda x: int(str(x)[0].isupper()))
    df['response_a_length'] = df['response_a'].str.len()
    df['response_b_length'] = df['response_b'].str.len()
    df['response_length_difference'] = abs(df['response_a_length'] - df['response_b_length'])

    # winner_encoded sütunu oluşturma
    le = LabelEncoder()
    df['winner_encoded'] = le.fit_transform(df['winner'])

    # model_name sütunu oluşturma
    df['model_name'] = np.where(df['winner_model_a'] == 1, df['model_a'], df['model_b'])

    # Kategorik değişkenleri encode etme
    categorical_columns = ['model_a', 'model_b', 'model_name']
    for col in categorical_columns:
        le = LabelEncoder()
        df[f'{col}_encoded'] = le.fit_transform(df[col])

    # Boolean değerleri 0 ve 1'e çevirme
    df['model_a_win'] = df['winner'] == 'model_a'
    df['model_b_win'] = df['winner'] == 'model_b'
    df['winner_tie'] = df['winner'] == 'tie'
    df['tie'] = df['winner'] == 'tie'

    boolean_columns = ['response_b_startswith_upper', 'response_a_startswith_upper', 'model_a_win', 'model_b_win', 'winner_tie', 'tie']
    for col in boolean_columns:
        df[col] = df[col].astype(int)

    return df

# Veri setini hazırlama
prepared_df = prepare_data_for_ml(df)

# Gerekli sütunları seçme ve eksik sütunları kontrol etme
columns_to_keep = [
    'id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b',
    'winner', 'model_a_win', 'model_b_win', 'winner_tie',
    'response_a_punctuation_count', 'response_b_punctuation_count',
    'response_b_startswith_upper', 'response_a_startswith_upper',  # response_a_startswith_upper eklendi
    'response_a_length', 'response_b_length',
    'response_length_difference', 'winner_encoded',
    'model_name', 'tie', 'model_a_encoded', 'model_b_encoded', 'model_name_encoded'
]

prepared_df = prepared_df[columns_to_keep]

# Veriyi hazırlama
X = prepared_df[['response_a_punctuation_count', 'response_b_punctuation_count',
        'response_a_startswith_upper', 'response_b_startswith_upper',
        'response_a_length', 'response_b_length', 'response_length_difference']]
y = prepared_df['winner_encoded']  # LabelEncoded kullanılarak kodlanmış hedef değişken

# Veriyi train ve test setlerine bölelim
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Veriyi standardize edelim
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modeli eğitme
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# Model üzerinde tahmin yapma
y_pred = model.predict(X_test_scaled)

# Sonuçları değerlendirme
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=['model_a', 'model_b', 'tie']))


In [ ]:
df['winner']

In [ ]:
import pandas as pd
import numpy as np
import string
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer

In [ ]:
# Kazanan modeli encode etmek
label_encoder = LabelEncoder()
df['winner_encoded'] = label_encoder.fit_transform(df['winner'])

# Numeric özellikler ve metinsel özellikler için kolonları ayırmak
numeric_features = [
    'response_a_punctuation_count', 'response_b_punctuation_count',
    'response_a_unique_words', 'response_b_unique_words',
    'response_a_stop_words_ratio', 'response_b_stop_words_ratio',
    'response_a_length', 'response_b_length', 'response_length_difference'
]

text_features = ['prompt', 'response_a', 'response_b']

# ColumnTransformer oluşturmak
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('text', 'passthrough', text_features)  # Metinsel özellikler passthrough olarak geçirilecek
    ])

# Verileri X (özellikler) ve y (hedef değişken) olarak ayırmak
X = df[numeric_features + text_features]
y = df['winner_encoded']

# Train ve test veri setlerini oluşturmak
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline oluşturmak
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Modeli eğitmek
pipeline.fit(X_train, y_train)

# Test veri seti üzerinde modeli değerlendirmek
accuracy = pipeline.score(X_test, y_test)
print(f"Model accuracy: {accuracy}")


In [ ]:
import pandas as pd

# CSV dosyalarını okuma
train_df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
test_df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# Birleştirme işlemi
df = pd.concat([train_df, test_df], ignore_index=True)

# Sonucu kontrol etme
df.head()


In [ ]:
df.shape

In [ ]:
len(df.id)

In [ ]:
df.info()

In [ ]:
winner_counts = {
    'model_a': df['winner_model_a'].value_counts(),
    'model_b': df['winner_model_b'].value_counts(),
    'tie': df['winner_tie'].sum()
}

print(winner_counts)

In [ ]:
# winner_model_a ve winner_model_b sütunları sıfır olan satırları filtreleme
zero_winner_df = df[(df['winner_model_a'] == 1.0) & (df['winner_model_b'] == 1.0)].astype("float64")

In [ ]:
# winner_model_a sütunu 1 olan satırları filtreleme
model_a_winner_df = combined_df[combined_df['winner_model_a'] == 1.0]

# winner_model_b sütunu 1 olan satırları filtreleme
model_b_winner_df = combined_df[combined_df['winner_model_b'] == 1.0]

# 1 olan satırlarda model_a ve model_b'nin en çok tekrar edileni bulma
model_a_counts = model_a_winner_df['model_a'].value_counts()
model_b_counts = model_b_winner_df['model_b'].value_counts()

print("model_a en çok tekrar edenler (combined):")
print(model_a_counts.head())

print("\nmodel_b en çok tekrar edenler (combined):")
print(model_b_counts.head())

In [ ]:
# model_a sütununda en çok tekrar eden yanıtları bulma
response_a_counts = model_a_winner_df['response_a'].value_counts()
response_b_counts = model_b_winner_df['response_b'].value_counts()

In [ ]:
# response_a sütununda en çok tekrar eden yanıtları bulma ve kazanan modele göre gruplandırma
response_a_counts_df = model_a_winner_df.groupby(['model_a', 'prompt']).size().reset_index(name='count')

# response_b sütununda en çok tekrar eden yanıtları bulma ve kazanan modele göre gruplandırma
response_b_counts_df = model_b_winner_df.groupby(['model_b', 'prompt']).size().reset_index(name='count')

print("response_a en çok tekrar edenler (combined):")
print(response_a_counts_df.head())

print("\nresponse_b en çok tekrar edenler (combined):")
print(response_b_counts_df.head())

In [ ]:
response_a_counts_df

In [ ]:
top_models = ['gpt-4-1106-preview', 'gpt-4-0613', 'gpt-3.5-turbo-0613', 'gpt-4-0314']


In [ ]:
# Her model için en çok tekrar eden 10 yanıtı almak
for model in top_models:
    print(f"\nModel: {model} - En çok tekrar eden 10 yanıt (response_a):")
    top_responses_a = response_a_counts_df[response_a_counts_df['model_a'] == model].sort_values(by='count', ascending=False).head(10)
    print(top_responses_a)

    print(f"\nModel: {model} - En çok tekrar eden 10 yanıt (response_b):")
    top_responses_b = response_b_counts_df[response_b_counts_df['model_b'] == model].sort_values(by='count', ascending=False).head(10)
    print(top_responses_b)

In [ ]:
# Kazanan modelin adını belirleyen ve yeni bir sütun oluşturan fonksiyon
def determine_winner(row):
    if row['winner_model_a'] == 1.0:
        return row['model_a']
    elif row['winner_model_b'] == 1.0:
        return row['model_b']
    else:
        return 'tie'

# Kazanan sütunu oluşturma
df['winner'] = df.apply(determine_winner, axis=1)

# Sadece prompt ve kazanan sütununu alarak gruplama işlemi
grouped_df = df[['prompt', 'winner']].groupby('prompt')['winner'].apply(list).reset_index()

# CSV olarak çıktı alma
grouped_df.to_csv('prompt_winners.csv', index=False)

print("CSV dosyası başarıyla oluşturuldu: 'prompt_winners.csv'")

In [ ]:
prompt = pd.read_csv("/kaggle/working/prompt_winners.csv")
prompt.groupby("winner").count().reset_index().head(20)

In [ ]:
# Promptları kazanan modele göre gruplayarak en çok tekrar edenleri bulma
winner_counts = prompt_df.explode('winner').groupby('winner').size().reset_index(name='count')
winner_counts = winner_counts.sort_values(by='count', ascending=False)

# CSV olarak çıktı alma
winner_counts.to_csv('prompt_top_winners.csv', index=False)

print("CSV dosyası başarıyla oluşturuldu: 'prompt_top_winners.csv'")

In [ ]:
winner_counts.head(20)

In [ ]:
# Prompt ve kazanan modeli gruplayarak çıktı alma
prompt_winners = df.groupby(['winner', 'prompt']).sum().reset_index()

prompt_winners

In [ ]:
top_prompt = pd.read_csv("/kaggle/working/prompt_top_winners.csv")
top_prompt.head(20)

In [ ]:
# 'tie' durumunda response_a ve response_b değerlerini gruplayarak çıktı alma
tie_responses = df[df['winner'] == 'tie'].groupby(['response_a', 'response_b']).sum().reset_index()[['response_a', 'response_b',"prompt"]]

tie_responses

In [ ]:
def determine_winner_and_loser(row):
    if row['winner_model_a'] == 1.0:
        winner = row['model_a']
        loser = row['model_b']
        winner_response = row['response_a']
        loser_response = row['response_b']
    elif row['winner_model_b'] == 1.0:
        winner = row['model_b']
        loser = row['model_a']
        winner_response = row['response_b']
        loser_response = row['response_a']
    else:
        winner = 'tie'
        loser = 'tie'
        winner_response = ''
        loser_response = ''
    
    return pd.Series([winner, loser, len(winner_response), len(loser_response)], index=['winner', 'loser', 'winner_response_length', 'loser_response_length'])

# Kazanan ve kaybeden sütunlarını oluşturma
winner_loser_lengths = df.apply(determine_winner_and_loser, axis=1)

# Önceki DataFrame'e kazanan ve kaybeden uzunluklarını ekleyerek yeni bir DataFrame oluşturma
prompt_df = pd.concat([df[['prompt']], winner_loser_lengths], axis=1)

prompt_df.head(20)

In [ ]:
# CSV dosyalarını okuma
train_df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
test_df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# Birleştirme işlemi
df = pd.concat([train_df, test_df], ignore_index=True)

In [ ]:
train_df

In [ ]:
df

In [ ]:
# Polars DataFrame oluşturma
import polars as pl
from textblob import TextBlob
import numpy as np

In [ ]:
import polars as pl
import numpy as np
from textblob import TextBlob

# Polars DataFrame oluşturma
df = pl.DataFrame(df)

# Sentiment analizi ve error sayımı fonksiyonları
def batch_analyze_sentiment(texts):
    return np.array([TextBlob(text).sentiment.polarity for text in texts])

def batch_count_errors(texts):
    return np.array([text.lower().count('error') for text in texts])

# Yeni sütunlar oluşturma
df = df.with_columns([
    pl.col('response_a').str.lengths().alias('response_a_length'),
    pl.col('response_b').str.lengths().alias('response_b_length'),
    (pl.col('response_a').str.lengths() - pl.col('response_b').str.lengths()).abs().alias('response_length_difference'),
])

# Sentiment analizi ve error sayımı için ilk 100 satırı işleme al
if len(df) > 0:
    n_rows = min(100, len(df))
    
    sentiment_a = batch_analyze_sentiment(df['response_a'].head(n_rows).to_numpy())
    sentiment_b = batch_analyze_sentiment(df['response_b'].head(n_rows).to_numpy())
    errors_a = batch_count_errors(df['response_a'].head(n_rows).to_numpy())
    errors_b = batch_count_errors(df['response_b'].head(n_rows).to_numpy())

    # Yeni sütunları ekleme
    df = df.with_columns([
        pl.Series('sentiment_a', sentiment_a).extend(pl.Series([None] * (len(df) - n_rows))),
        pl.Series('sentiment_b', sentiment_b).extend(pl.Series([None] * (len(df) - n_rows))),
        pl.Series('errors_a', errors_a).extend(pl.Series([None] * (len(df) - n_rows))),
        pl.Series('errors_b', errors_b).extend(pl.Series([None] * (len(df) - n_rows))),
    ])

# Kazanan ve kaybeden sütunlarını oluşturma (eski sürümlerle uyumlu)
df = df.with_columns([
    pl.when(pl.col('sentiment_a') > pl.col('sentiment_b'))
      .then(pl.col('response_a'))
      .when(pl.col('sentiment_a') <= pl.col('sentiment_b'))
      .then(pl.col('response_b'))
      .alias('winner_sentiment'),
    pl.when(pl.col('sentiment_a') < pl.col('sentiment_b'))
      .then(pl.col('response_a'))
      .when(pl.col('sentiment_a') >= pl.col('sentiment_b'))
      .then(pl.col('response_b'))
      .alias('loser_sentiment')
])

df